In [1]:
import torch
from data_reading import WindowDataset
import os
import cv2
import numpy as np
from scipy import signal
import pickle

In [2]:
image_width = 128
image_height = 96
labels_folder = '../data/labels/'
imgs_folder = '../data/images/'
video_folder = '../data/training_videos/'
spectr_folder = '../data/spectrograms/'

In [3]:
dataset = WindowDataset(video_folder, labels_folder, imgs_folder)

In [4]:
dataset.save(spectr_folder)

29.866666666666667
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(8, 3, 9, 12)
(

In [ ]:
all_spectr = {}
for file in os.listdir(spectr_folder):
    if '.spectr' in file:
        name = file.split('.')[0]
        with open(f'{spectr_folder}{file}') as f:
            spectr = pickle.load(f)
        all_spectr[name] = spectr